In [1]:
from data_processing import import_and_clean_train, one_hot_encoding, label_encoding


train_x_le, test_x_le, train_y_le, test_y_le, label_dict = import_and_clean_train('train', label_encoding)
train_x_ohe, test_x_ohe, train_y_ohe, test_y_ohe = import_and_clean_train('train', one_hot_encoding)



SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.472614
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.349876
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
GarageYrBlt      0.261366
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
KitchenAbvGr     0.135907
EnclosedPorch    0.128578
ScreenPorch      0.111447
PoolArea         0.092404
MSSubClass       0.084284
OverallCond      0.077856
MoSold           0.046432
3SsnPorch        0.044584
YrSold           0.028923
LowQualFinSF     0.025606
MiscVal          0.021190
BsmtHalfBath     0.016844
BsmtFinSF2       0.011378
Name: SalePrice, dtype: float64
SalePr

In [2]:
# Imports
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error
import seaborn as sns



# Creating forests with label
regr_le = RandomForestRegressor(n_jobs=-1)
regr_le.fit(train_x_le, train_y_le)
# Getting prediction and calculating accuracy
y_prediction_forest_le = regr_le.predict(test_x_le)
train_prediction_forest_le = regr_le.predict(train_x_le)

# Creating forests with one_hot
regr_ohe = RandomForestRegressor(n_jobs=-1)
regr_ohe.fit(train_x_ohe, train_y_ohe)
# Getting prediction and calculating accuracy
y_prediction_forest_ohe = regr_ohe.predict(test_x_ohe)
train_prediction_forest_ohe = regr_ohe.predict(train_x_ohe)




# Calculate average deviation
def dev(y, y_hat):
    tot = 0
    for i in range(len(y)):
        tot += abs(y[i]-y_hat[i])
    return tot / len(y)


# Forest ohe
print('Forest ohe')
print('test', dev(test_y_le, y_prediction_forest_ohe), mean_squared_log_error(test_y_le, y_prediction_forest_ohe))
print('train', dev(train_y_le, train_prediction_forest_ohe), mean_squared_log_error(train_y_le, train_prediction_forest_ohe), '\n')

# Forest le
print('Forest le')
print('test', dev(test_y_le, y_prediction_forest_le), mean_squared_log_error(test_y_le, y_prediction_forest_le))
print('train', dev(train_y_le, train_prediction_forest_le), mean_squared_log_error(train_y_le, train_prediction_forest_le))

Forest ohe
test 15430.947694063925 0.01709863395393699
train 6817.49295499021 0.0039563471844107296 

Forest le
test 15789.5462785388 0.018277932700984177
train 6697.2240998043035 0.0037839907534065733


In [3]:
#random_grid = {'n_estimators': [700, 900, 1100, 1300], 'max_depth': [15, 20, 25, 30, 35], 'max_features': [0.2, 0.3, 0.4, 0.5, 0.6, 'sqrt', 'log2'], 'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4], 'bootstrap': [False]}

#random_search = RandomizedSearchCV(estimator = regr_le, param_distributions = random_grid, n_jobs=-1, cv=20)
#random_search.fit(train_x_le, train_y_le)
#print(random_search.best_params_)

In [4]:
#search_grid = {'n_estimators': [1100, 1300], 'max_depth': [25, 30, 35], 'max_features': [0.3, 0.4, 0.5], 'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3], 'bootstrap': [False]}
#search_grid2 = {'n_estimators': [1100], 'max_depth': [35], 'max_features': [0.3], 'min_samples_leaf': [1], 'min_samples_split': [2], 'bootstrap': [False], 'min_impurity_decrease':[0, 0.1, 0.2, 0.3]}

#grid_search = GridSearchCV(estimator= regr_le, param_grid=search_grid2, n_jobs=-1)
#grid_search.fit(train_x_le, train_y_le)
#print(grid_search.best_params_)

In [5]:
n = 20

y_pred = 0
train_pred = 0
y_pred_log = 0
train_pred_log = 0


for i in range(n):
    # Creating forests with label and the best parameters
    regr_le_random = RandomForestRegressor(max_depth= 35, max_features= 0.3, min_samples_leaf= 1, n_estimators= 1100, min_samples_split=2, bootstrap=False, n_jobs=-1)
    regr_le_random.fit(train_x_le, train_y_le)
    # Getting prediction and calculating accuracy
    y_prediction_forest_le_random = regr_le_random.predict(test_x_le)
    train_prediction_forest_le_random = regr_le_random.predict(train_x_le)
    
    y_pred += dev(test_y_ohe, y_prediction_forest_le_random) 
    train_pred += dev(train_y_ohe, train_prediction_forest_le_random) 

    y_pred_log += mean_squared_log_error(test_y_ohe, y_prediction_forest_le_random) 
    train_pred_log += mean_squared_log_error(train_y_ohe, train_prediction_forest_le_random) 

# Forest le grid
print('Forest le random')
print('test', y_pred/n, y_pred_log/n)
print('train', train_pred/n, train_pred_log/n, '\n')

Forest le random
test 14244.881873702785 0.015558173641990048
train 0.0036915139654886396 7.056335663037238e-13 



In [6]:
y_pred = 0
train_pred = 0
y_pred_log = 0
train_pred_log = 0


for i in range(n):
    # Creating forests with label and the best parameters
    regr_le_random = RandomForestRegressor(max_depth= 20, max_features= 0.3, min_samples_leaf= 1, n_estimators= 1100, min_samples_split=2, bootstrap=False, n_jobs=-1)
    regr_le_random.fit(train_x_ohe, train_y_ohe)
    # Getting prediction and calculating accuracy
    y_prediction_forest_le_random = regr_le_random.predict(test_x_ohe)
    train_prediction_forest_le_random = regr_le_random.predict(train_x_ohe)
    
    y_pred += dev(test_y_ohe, y_prediction_forest_le_random) 
    train_pred += dev(train_y_ohe, train_prediction_forest_le_random) 

    y_pred_log += mean_squared_log_error(test_y_ohe, y_prediction_forest_le_random) 
    train_pred_log += mean_squared_log_error(train_y_ohe, train_prediction_forest_le_random) 

# Forest le grid
print('Forest ohe random')
print('test', y_pred/n, y_pred_log/n)
print('train', train_pred/n, train_pred_log/n, '\n')

Forest ohe random
test 14199.176723411938 0.015357769108909649
train 21.23852048854322 8.431501682187225e-08 

